# MALLORN investigations - data processing and ML testing
31/01/26 comment: This is a highly reduced version of a notebook with many experiments. Only the necessary outcomes for my final solution are kept here. They are:<br>
- Adding a de-extinguished flux column ("Fluxcorr") to raw light curve files
- Extracting SALT2 parameters and add to updated logs



In [2]:
# Imports
# Basic
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Extinction
from extinction import fitzpatrick99
# Signal processing functions
from scipy.signal import find_peaks
from scipy.signal import peak_prominences

import seaborn as sns

#Defining function to de-extinct a set of flux values
def jurassic_park (flux, eff_wl):
    A_lambda = fitzpatrick99(eff_wl, ebv * 3.1) #3.1 = Standard Milky Way value
    flux_ext = flux * 10**((A_lambda)/2.5)
    return flux_ext, A_lambda

#Defining a function to convert from AB magnitude to flux in microjanskys
def ab_to_uJy (magAB):
    flux_Jy = 10**(23 - (magAB + 48.6)/2.5)
    flux_mJy = flux_Jy*1000000
    return flux_mJy

#Defining a function to convert from flux in microjanskys to AB magnitude
def uJy_to_ab (mJy):
    flux_Jy = mJy/1000000
    ab = 2.5*(23 - np.log10(flux_Jy))-48.6
    return ab

## Data definitions

In [784]:
# List of splits
splits=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
#Effective wavelength for each band - sourced from SVO Filter Profile Service
u_eff_wl = np.array([3641]); g_eff_wl = np.array([4704]); r_eff_wl = np.array([6155])
i_eff_wl = np.array([7504]); z_eff_wl = np.array([8695]); y_eff_wl = np.array([10056])

## De-extinguish LCs from MW dust

In [781]:
# WIP!!
for split in splits:
    fpath='content/mallorn-astronomical-classification-challenge/'
    # training data
    #df=pd.read_csv(fpath+'split_{}/train_full_lightcurves.csv'.format(split))
    # test
    df=pd.read_csv(fpath+'split_{}/test_full_lightcurves.csv'.format(split))
    df['Fluxcorr']=0
    # log
    #log=pd.read_csv(fpath+'train_log.csv')
    log=pd.read_csv(fpath+'test_log.csv')
    # object ids in training data set
    obj_id=df.object_id.unique()
    # get ebv from log file
    for object_ID in obj_id:
        #Creating masks to isolate data for chosen lightcurve and split according to filter
        u_mask = ((df['Filter'] == 'u') & (df['object_id'] == object_ID))
        g_mask = ((df['Filter'] == 'g') & (df['object_id'] == object_ID))
        r_mask = ((df['Filter'] == 'r') & (df['object_id'] == object_ID))
        i_mask = ((df['Filter'] == 'i') & (df['object_id'] == object_ID))
        z_mask = ((df['Filter'] == 'z') & (df['object_id'] == object_ID))
        y_mask = ((df['Filter'] == 'y') & (df['object_id'] == object_ID))
        
        #Saving fluxes, times and error values for the object split according to filter
        u_flux = np.array(df.loc[u_mask, 'Flux']); u_time = np.array(df.loc[u_mask, 'Time (MJD)']); u_err = np.array(df.loc[u_mask, 'Flux_err'])
        g_flux = np.array(df.loc[g_mask, 'Flux']); g_time = np.array(df.loc[g_mask, 'Time (MJD)']); g_err = np.array(df.loc[g_mask, 'Flux_err'])
        r_flux = np.array(df.loc[r_mask, 'Flux']); r_time = np.array(df.loc[r_mask, 'Time (MJD)']); r_err = np.array(df.loc[r_mask, 'Flux_err'])
        i_flux = np.array(df.loc[i_mask, 'Flux']); i_time = np.array(df.loc[i_mask, 'Time (MJD)']); i_err = np.array(df.loc[i_mask, 'Flux_err'])
        z_flux = np.array(df.loc[z_mask, 'Flux']); z_time = np.array(df.loc[z_mask, 'Time (MJD)']); z_err = np.array(df.loc[z_mask, 'Flux_err'])
        y_flux = np.array(df.loc[y_mask, 'Flux']); y_time = np.array(df.loc[y_mask, 'Time (MJD)']); y_err = np.array(df.loc[y_mask, 'Flux_err'])
    
        #Selecting only the data relevant to the chosen object
        object_log = log.loc[log['object_id'] == object_ID]
        
        #Defining the redshift, ebv and type of that object
        redshift = object_log['Z']
        ebv = object_log['EBV']
        SpecType = object_log['SpecType']
    
        #De-extinguish and add back to df
        u_flux, u_A_lambda = jurassic_park(u_flux,u_eff_wl); g_flux, g_A_lambda = jurassic_park(g_flux,g_eff_wl)
        r_flux, r_A_lambda = jurassic_park(r_flux,r_eff_wl); i_flux, i_A_lambda = jurassic_park(i_flux,i_eff_wl)
        z_flux, z_A_lumbda = jurassic_park(z_flux,z_eff_wl) ;y_flux, y_A_lambda = jurassic_park(y_flux,y_eff_wl)
        df.loc[u_mask,'Fluxcorr']=u_flux; df.loc[g_mask,'Fluxcorr']=g_flux
        df.loc[r_mask,'Fluxcorr']=r_flux; df.loc[i_mask,'Fluxcorr']=i_flux
        df.loc[z_mask,'Fluxcorr']=z_flux; df.loc[y_mask,'Fluxcorr']=y_flux
    df.to_csv(fpath+'split_{}/test_full_lightcurves.csv'.format(split))
    #df.to_csv(fpath+'split_{}/train_full_lightcurves.csv'.format(split))

        

/var/folders/z1/c7w4xcjn3c7479pfs85459zc0000z9/T/ipykernel_92843/2348710287.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  A_lambda = fitzpatrick99(eff_wl, ebv * 3.1) #3.1 = Standard Milky Way value
/var/folders/z1/c7w4xcjn3c7479pfs85459zc0000z9/T/ipykernel_92843/2897567868.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 0.31973824 -0.38650459  0.10297634  1.47708944 -0.19844455  0.39186723
 -0.56102123 -0.20790577 -0.192507    0.74781627  0.94225281 -0.00588706]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[u_mask,'Fluxcorr']=u_flux; df.loc[g_mask,'Fluxcorr']=g_flux
/var/folders/z1/c7w4xcjn3c7479pfs85459zc0000z9/T/ipykernel_92843/2348710287.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError 

## Extract SALT2 parameters via sncosmo

In [841]:
import sncosmo as sn
from astropy.table import Table
# Load log
df_log=pd.read_csv('content/mallorn-astronomical-classification-challenge/test_log_updated.csv')
#df_log=pd.read_csv('content/mallorn-astronomical-classification-challenge/train_log_updated.csv')
splits=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
# Filter dictionary
filt_dict={'u':'lsstu','g':'lsstg','r':'lsstr','i':'lssti','z':'lsstz','y':'lssty'}
col_dict={'Time (MJD)':'time','Filter':'band','Flux':'flux','Flux_err':'fluxerr'}
# Define SALT2 model
dust=sn.F99Dust()
model = sn.Model(source='salt2', effects=[dust], effect_names=['mw'], effect_frames=['obs'])
# Define new feature columns
col_names = ['x1_salt2','c_salt2','chisq_salt2','redchisq_salt2']
df_log[col_names] = np.nan

# sncosmo input columns
salt_cols=['time','band','flux','fluxerr','zp','zpsys']

for obj_id in df_log['object_id']:
    split=df_log.loc[df_log.object_id==obj_id,'split'].values[0]
    redshift=df_log.loc[df_log.object_id==obj_id,'Z'].values[0]
    ebv=df_log.loc[df_log.object_id==obj_id,'EBV'].values[0]
    df=pd.read_csv('content/mallorn-astronomical-classification-challenge/{}/test_full_lightcurves.csv'.format(split))
    #df=pd.read_csv('content/mallorn-astronomical-classification-challenge/{}/train_full_lightcurves.csv'.format(split))
    # Include only dps with snr>=3
    mask = ((df['object_id'] == obj_id)&(df['Flux']/df['Flux_err']>=3))
    df_salt=df[mask].copy()
    df_salt['zp']=25
    df_salt['zpsys']='ab'
    df_salt.rename(columns=col_dict,inplace=True)
    df_salt.replace(filt_dict,inplace=True)
    df_salt=df_salt[salt_cols]
    #print(df_salt)

    # SALT2 model
    model.set(z=redshift,mwebv=ebv)
    table_salt = Table.from_pandas(df_salt)
    try:
        result, fitted_model = sn.fit_lc(table_salt, model, ['t0', 'x0', 'x1', 'c'],minsnr=3)
        c=result.parameters[4]
        x1=result.parameters[3]
        chisq=result.chisq
        red_chi=result.chisq/result.ndof
        df_log.loc[df_log.object_id==obj_id,col_names]=[x1,c,chisq,red_chi]
    except Exception as e:
        print('salt2 fit failed for {}, cols are nan'.format(obj_id))
df_log.to_csv('content/mallorn-astronomical-classification-challenge/test_log_updated.csv')
#df_log.to_csv('content/mallorn-astronomical-classification-challenge/train_log_updated.csv')

/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lssty(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu, lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu, lsstg(out 

salt2 fit failed for nimp_luin_gellui, cols are nan


/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu, lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu, lsstr, lss

salt2 fit failed for rom_amloth_Rochirrim, cols are nan


/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lssty(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/models.py:813: RuntimeWarning: overflow encountered in power
  10. ** (-0.4 * self._colorlaw(wave) * self._parameters[2]))
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/models.py:812: RuntimeWarning: invalid value encountered in multiply
  return (self._parameters[0] * (m0 + self._parameters[1] * m1) *
/opt/anaconda3/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/anaconda3/lib/pytho

...

/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lssty(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu, lsstr, lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstg(out 

salt2 fit failed for aduial_bass_espalass, cols are nan


/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstr, lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu, lsstr, lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +


salt2 fit failed for amarth_onodrim_gwathui, cols are nan


/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu, lsstg(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lssty(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/fitting.py:191: RuntimeWarning: Dropping following bands from data: lsstu(out of model wavelength range)
  warnings.warn("Dropping following bands from data: " +
/opt/anaconda3/lib/python3.13/site-packages/sncosmo/models.py:813: RuntimeWarning: overflow encountered in power
  10. ** (-0.4 * self._c